# Scrapping Yelp

In [2]:
import requests
import pandas as pd
import time

In [3]:
api_key_rose = "v6SzGq_aIaieBTKok8x3AvMLP8ffdkmhMPB5ZpdKPF5eJPbiQRCFA7z4kQXssptbvOLizaX6eMxYEdNwlwyhaRgPeXagY9pPH0x-sSkKsUUm2sGbUl1R_S9I5pymZXYx"

def get_restaurants(api_key, location, total=500):
    headers = {"Authorization": f"Bearer {api_key}"}
    url = "https://api.yelp.com/v3/businesses/search"
    restaurants = []
    offset = 0
    limit = 50  # Le maximum autorisé par Yelp

    while len(restaurants) < total:
        params = {
            "term": "restaurant",
            "location": location,
            "limit": limit,
            "offset": offset
        }
        response = requests.get(url, headers=headers, params=params)
        if response.status_code != 200:
            break
        data = response.json()
        restaurants.extend(data.get("businesses", []))
        offset += limit
        time.sleep(0.5)  # Pour éviter de dépasser la limite de taux

    return restaurants[:total]

restaurants = get_restaurants(api_key_rose, "Paris, France")



In [4]:
def get_reviews(api_key, business_id, total=20):
    headers = {"Authorization": f"Bearer {api_key}"}
    url = f"https://api.yelp.com/v3/businesses/{business_id}/reviews"
    params = {"limit": total}
    response = requests.get(url, headers=headers, params=params)
    if response.status_code != 200:
        return []
    data = response.json()
    return data.get("reviews", [])

for restaurant in restaurants:
    business_id = restaurant['id']
    reviews = get_reviews(api_key_rose, business_id)
    restaurant['reviews'] = reviews
    time.sleep(0.5)  # Pour éviter de dépasser la limite de taux


In [5]:
print(restaurants)

[{'id': '-0iLH7iQNYtoURciDpJf6w', 'alias': 'le-comptoir-de-la-gastronomie-paris', 'name': 'Le Comptoir de la Gastronomie', 'image_url': 'https://s3-media3.fl.yelpcdn.com/bphoto/xT4YkCm_cZWbKbz9AVEnaA/o.jpg', 'is_closed': False, 'url': 'https://www.yelp.com/biz/le-comptoir-de-la-gastronomie-paris?adjust_creative=1UHFgqUYrECRVnlde3Ir-w&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=1UHFgqUYrECRVnlde3Ir-w', 'review_count': 1304, 'categories': [{'alias': 'french', 'title': 'French'}], 'rating': 4.5, 'coordinates': {'latitude': 48.8645157999652, 'longitude': 2.34540185646608}, 'transactions': [], 'price': '€€', 'location': {'address1': '34 rue Montmartre', 'address2': '', 'address3': '', 'city': 'Paris', 'zip_code': '75001', 'country': 'FR', 'state': '75', 'display_address': ['34 rue Montmartre', '75001 Paris', 'France']}, 'phone': '+33142333132', 'display_phone': '+33 1 42 33 31 32', 'distance': 370.8275165771047, 'reviews': [{'id': 'sxEFkJ89kyF-wMDUI2ZnWw', 'url': '

In [8]:
# Convertir les données des restaurants en DataFrame
restaurants_df = pd.DataFrame(restaurants)
restaurants_df.head()

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance,reviews
0,-0iLH7iQNYtoURciDpJf6w,le-comptoir-de-la-gastronomie-paris,Le Comptoir de la Gastronomie,https://s3-media3.fl.yelpcdn.com/bphoto/xT4YkC...,False,https://www.yelp.com/biz/le-comptoir-de-la-gas...,1304,"[{'alias': 'french', 'title': 'French'}]",4.5,"{'latitude': 48.8645157999652, 'longitude': 2....",[],€€,"{'address1': '34 rue Montmartre', 'address2': ...",+33142333132,+33 1 42 33 31 32,370.827517,"[{'id': 'sxEFkJ89kyF-wMDUI2ZnWw', 'url': 'http..."
1,IU9_wVOGBKjfqTTpAXpKcQ,bistro-des-augustins-paris,Bistro des Augustins,https://s3-media2.fl.yelpcdn.com/bphoto/ctHDHM...,False,https://www.yelp.com/biz/bistro-des-augustins-...,486,"[{'alias': 'bistros', 'title': 'Bistros'}, {'a...",4.5,"{'latitude': 48.854754, 'longitude': 2.342119}",[],€€,"{'address1': '39 quai des Grands Augustins', '...",+33143540441,+33 1 43 54 04 41,801.117610,"[{'id': 'PJuWhEzKFz3ipwhOcWMMBA', 'url': 'http..."
2,cEjF41ZQB8-SST8cd3EsEw,l-avant-comptoir-paris-3,L'Avant Comptoir,https://s3-media2.fl.yelpcdn.com/bphoto/V38oU4...,False,https://www.yelp.com/biz/l-avant-comptoir-pari...,657,"[{'alias': 'tapas', 'title': 'Tapas Bars'}, {'...",4.5,"{'latitude': 48.85202, 'longitude': 2.3388}",[],€€,"{'address1': '3 carrefour de l'Odéon', 'addres...",+33142384755,+33 1 42 38 47 55,1131.333887,"[{'id': 'WiSWp2PnMqvPyNMkH9n-rQ', 'url': 'http..."
3,BuJnfWI86tTxFUon071EKg,brasserie-bellanger-paris,Brasserie Bellanger,https://s3-media2.fl.yelpcdn.com/bphoto/0IQnl-...,False,https://www.yelp.com/biz/brasserie-bellanger-p...,24,"[{'alias': 'french', 'title': 'French'}]",5.0,"{'latitude': 48.880937, 'longitude': 2.3499401}",[],NaN,{'address1': '140 rue du Faubourg Poissonnière...,+33954009965,+33 9 54 00 99 65,2185.842371,"[{'id': 'id68lPNLI6S7aaKhurdHxQ', 'url': 'http..."
4,ctP4c3mwVO5oOzLI48LtuQ,les-antiquaires-paris,Les Antiquaires,https://s3-media1.fl.yelpcdn.com/bphoto/5NWC7V...,False,https://www.yelp.com/biz/les-antiquaires-paris...,468,"[{'alias': 'bistros', 'title': 'Bistros'}]",4.5,"{'latitude': 48.8580664639432, 'longitude': 2....",[],€€€,"{'address1': '13 rue du Bac', 'address2': '', ...",+33142610836,+33 1 42 61 08 36,1105.856028,"[{'id': 'lXP0qW8tHuJa9C0ZIvzhPQ', 'url': 'http..."


In [12]:
# Créer un DataFrame pour les avis
reviews_data = []
for restaurant in restaurants:
    for review in restaurant.get('reviews', []):
        review_data = {
            'business_id': restaurant['id'],
            'review_id': review['id'],
            'text': review['text'],
            'rating': review['rating'],
            'time_created': review['time_created'],
            'user_id': review['user']['id'] if review.get('user') else None
        }
        reviews_data.append(review_data)

reviews_df = pd.DataFrame(reviews_data)

# Enregistrer les DataFrames en fichiers CSV
restaurants_df.to_csv('restaurants1.csv', index=False)
reviews_df.to_csv('reviews1.csv', index=False)
